# RAG(검색-증강 생성) Retrieval-Augmented Generation
- LLM이 외부 데이터를 컨텍스트로 활용할 수 있도록..

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ), # 하나의 청크...
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]

In [4]:
%pip install -q "langchain_chroma>=0.1.2" langchain_community faiss-cpu

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
from langchain_openai import OpenAIEmbeddings # 숫자로 바꿔줄 거임
from langchain_chroma import Chroma

# OpenAI 임베딩 모델
embedding = OpenAIEmbeddings(model='text-embedding-3-small')

# x = embedding.embed_query('점심 너무 많이 먹었다 정말 정말 졸림 진짜')
# print(len(x))

# VectorStore에 임베딩 후 저장
vectorstore = Chroma.from_documents(
    documents,
    embedding=embedding
)

In [15]:
vectorstore.similarity_search(
    'bonito', k=2
)

[Document(id='3f2c5957-5a49-4a8b-bc77-15dbda96ab6d', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
 Document(id='b4c39251-4b1e-4ecb-b93b-32b48fffbf2f', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.')]

In [16]:
%pip install -q pymupdf

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## `사전처리 단계`
1. 문서 Loading
1. 텍스트 나누기 (Splitting)
1. 숫자로 바꾸기 (Embedding)
1. 저장하기 (VectorStore)

In [26]:
# 1. Load

from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader('./data/spri.pdf')
docs = loader.load()

# for doc in docs:
#     print(doc)

print('원본 pdf 장:', len(docs))

원본 pdf 장: 23


In [ ]:
# 2. Split

from langchain_text_splitters import RecursiveCharacterTextSplitter # 가장 많이 씀

# 500글자당 하나의 chunk, 50글자는 겹치게 나눈당
text_spliter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = text_spliter.split_documents(docs)

print('split한 pdf 장수:', len(split_docs))

for doc in split_docs:
    print('==============')
    print(doc)

split한 pdf 장수: 72
page_content='2023년 12월호' metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:28:38+09:00', 'source': './data/spri.pdf', 'file_path': './data/spri.pdf', 'total_pages': 23, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'moddate': '2023-12-08T13:28:38+09:00', 'trapped': '', 'modDate': "D:20231208132838+09'00'", 'creationDate': "D:20231208132838+09'00'", 'page': 0}
page_content='2023년 12월호
Ⅰ. 인공지능 산업 동향 브리프
 1. 정책/법제 
   ▹ 미국, 안전하고 신뢰할 수 있는 AI 개발과 사용에 관한 행정명령 발표  ························· 1
   ▹ G7, 히로시마 AI 프로세스를 통해 AI 기업 대상 국제 행동강령에 합의··························· 2
   ▹ 영국 AI 안전성 정상회의에 참가한 28개국, AI 위험에 공동 대응 선언··························· 3
   ▹ 미국 법원, 예술가들이 생성 AI 기업에 제기한 저작권 소송 기각····································· 4
   ▹ 미국 연방거래위원회, 저작권청에 소비자 보호와 경쟁 측면의 AI 의견서 제출················· 5' metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationd

In [ ]:
# 3. 벡터 임베딩  4. 벡터 스토어 저장
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS

embedding = OpenAIEmbeddings()

vectorstore = FAISS.from_documents(documents=split_docs, embedding=embedding)


# test
vectorstore.similarity_search('미국 대통령과 관련된 문서들 가져와봐', k=4)

[Document(id='c082709e-b765-4f38-b97d-4a4f170d35f1', metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:28:38+09:00', 'source': './data/spri.pdf', 'file_path': './data/spri.pdf', 'total_pages': 23, 'format': 'PDF 1.4', 'title': '', 'author': 'dj', 'subject': '', 'keywords': '', 'moddate': '2023-12-08T13:28:38+09:00', 'trapped': '', 'modDate': "D:20231208132838+09'00'", 'creationDate': "D:20231208132838+09'00'", 'page': 18}, page_content='문법 교정기(그래머리), 생성 이미지 모델(달리2)\n미달성\n4단계: 거장(숙련된 인간의 99% 이상) \n딥블루, 알파고\n미달성\n5단계: 초인간(인간을 100% 능가)\n알파폴드, 알파제로, 스톡피시\n미달성\n<구글 딥마인드의 범용 AI 분류 프레임워크> \n☞ 출처 : Arxiv.org, Levels of AGI: Operationalizing Progress on the Path to AGI, 2023.11.04.'),
 Document(id='479c4501-3ac5-49a4-947a-b513df63ff49', metadata={'producer': 'Hancom PDF 1.3.0.542', 'creator': 'Hwp 2018 10.0.0.13462', 'creationdate': '2023-12-08T13:28:38+09:00', 'source': './data/spri.pdf', 'file_path': './data/spri.pdf', 'total_page

## `검색 증강 단계`
1. 사용자 질문 (Query)
1. 검색 (Retrieve)
1. LLM
1. 최종 답변

In [36]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core. runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

from langchain import hub

# Prompt 세팅
prompt = hub.pull('rlm/rag-prompt')

# LLM 모델
llm = ChatOpenAI(model='gpt-4.1-nano')

# 검색기 생성 (retriever 생성)
retriever = vectorstore.as_retriever()

chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | prompt | llm | StrOutputParser()
)

chain.invoke('삼성전자 관련 소식 다 가져와')

"삼성전자는 자체 개발한 생성 AI '삼성 가우스'를 공개했으며, 언어, 코드, 이미지 모델로 구성되어 있습니다. 이 AI는 온디바이스에서 작동하며 사용자 정보 유출 없이 안전하게 학습되었고, 다양한 제품에 단계적으로 탑재될 계획입니다. 2024년에는 가우스를 탑재한 삼성 스마트폰이 경쟁 기기들과 시장에서 경쟁할 것으로 기대됩니다."

In [38]:
%pip install -q beautifulsoup4 lxml

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# 관심있는 데이터 소스로 RAG 써보기
# 1. 회사 홈페이지!!!!!!!!!!
import requests
from bs4 import BeautifulSoup # 웹 페이지(HTML, XML)를 쉽게 파싱(Parsing) 해서 원하는 정보를 뽑아내는 라이브러리
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains import RetrievalQA

# 1. 크롤링
url = 'https://www.avatye.com/'
res = requests.get(url)
soup = BeautifulSoup(res.text, "html.parser")

# 텍스트만 추출
text = soup.get_text(separator="\n", strip=True)

# 2. 문서 쪼개기
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = splitter.create_documents([text])

# 3. Vector DB
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = Chroma.from_documents(docs, embeddings)

# 4. RAG Chain
retriever = vectorstore.as_retriever()
llm = ChatOpenAI(model="gpt-4o-mini")
qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever)

# 5. 실행
print(qa_chain.run("이 회사의 주요 사업 영역을 알려줘"))

C:\Users\agnes\AppData\Local\Temp\ipykernel_19904\3081674800.py:31: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(qa_chain.run("이 회사의 주요 사업 영역을 알려줘"))


이 회사의 주요 사업 영역은 모바일 앱에 리워드 광고를 연동하여 추가 수익을 창출하는 것입니다. 이를 위해 캐시버튼과 캐시룰렛 SDK를 제공하여 개발사와 퍼블리셔가 광고를 통해 수익을 얻을 수 있도록 지원하고 있습니다. 유저는 광고 시청을 통해 보상을 받을 수 있으며, 개발사는 기존 배너 광고 수익의 두 배 이상의 수익을 기대할 수 있습니다.


In [ ]:
# 사이트맵 있음	sitemap.xml 파싱으로 한 번에 URL 수집
# 사이트맵 없음	웹 크롤러 방식으로 링크 재귀적으로 추출

import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import time

def crawl_site(base_url, max_pages=100):
    visited = set([base_url])
    to_visit = [base_url]
    all_links = []

    while to_visit and len(visited) < max_pages:
        url = to_visit.pop(0)
        try:
            resp = requests.get(url)
            soup = BeautifulSoup(resp.text, "html.parser")
            for a in soup.find_all("a", href=True):
                href = urljoin(base_url, a['href'])
                parsed = urlparse(href)
                href = parsed.scheme + "://" + parsed.netloc + parsed.path
                if parsed.netloc == urlparse(base_url).netloc and href not in visited:
                    visited.add(href)
                    to_visit.append(href)
                    all_links.append(href)
            time.sleep(0.5)  # 서버에 부담 줄이기
        except Exception as e:
            print(f"Error crawling {url}: {e}")
    return all_links

links = crawl_site("https://www.avatye.com/", max_pages=50)
print(f"크롤링된 링크 수: {len(links)}")
print("\n".join(links))

크롤링된 링크 수: 5
https://www.avatye.com/product/cashbutton
https://www.avatye.com/product/cashroulette
https://www.avatye.com/resource
https://www.avatye.com/support
https://www.avatye.com/company


In [56]:
print(qa_chain.run('어떤 회사인지 알려줘'))

이 회사는 모바일 앱 개발사 및 운영사를 위한 리워드 시스템을 제공하는 회사입니다. 캐시버튼 SDK와 캐시룰렛 SDK를 통해 앱에 간단하게 연동하여 수익을 창출하고 사용자 경험을 향상시킬 수 있는 서비스를 제공합니다. 주요 서비스로는 사용자에게 광고 노출을 자율적으로 제어할 수 있게 하여 광고에 대한 보상을 제공하고, 개발사는 추가 수익을 얻을 수 있도록 하는 것입니다.


# Agent + RAG

In [52]:

from langchain.agents import create_openai_tools_agent, AgentExecutor # 에이전트 실행기
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory # 기억
from datetime import datetime
from langchain_tavily import TavilySearch

# RAG 관련
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.tools.retriever import create_retriever_tool


today = datetime.today().strftime('%Y-%m-%d')

llm = ChatOpenAI(model='gpt-4.1-nano')

search_tool = TavilySearch(
    max_results=5,
    topic='general'
)

loader = PyMuPDFLoader('./data/spri.pdf')
docs = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
split_docs = splitter.split_documents(docs)
embedding = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(split_docs, embedding=embedding) # 벡터스토어=저장소
retriever = vectorstore.as_retriever() # 리트리버=도서관 사서

rag_tool = create_retriever_tool(
    retriever,
    name='pdf_search',
    description='PDF 문서에서 질문과 관련된 내용을 검색합니다.' # Agent가 언제 이 tool을 쓸지 알게 됨
)


text = f'''
너는 웹 검색이 가능하고, 2023년 12월 인공지능 산업 최신동향 정보를 담은 pdf를 검색할 수 있는 어시스트야.
- 사용자가 pdf 문서와 관련된 질문(ex. '이 pdf에서', '문서에서', '파일에서')을 하면 반드시 'pdf_search' 도구를 써.
- 사용자 질문이 팩트체크를 필요로하고, 최신성이 필요하다 판단되면 web_search를 실행해.
- 사용자가 일반적인 질문을 하고, 최신성이나 팩트체크가 필요 없으면 그냥 답변해.
- 뭔가 확실하지 않으면, pdf_search와 web_search를 모두 실행해서 답변해.
- 오늘은 {today}야.
'''

prompt = ChatPromptTemplate.from_messages([
    ('system', text),
    MessagesPlaceholder(variable_name='chat_history'), # 사용자가 여태까지 했던 얘기들이 들어옴
    ('human', '{input}'),
    MessagesPlaceholder(variable_name='agent_scratchpad') #  임시저장 변수.... 검색 도구 호출 때 필요함
])

memory = ConversationBufferMemory(
    return_messages=True,
    memory_key='chat_history'
)

agent = create_openai_tools_agent(
    llm=llm,
    tools=[search_tool, rag_tool],
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=agent, 
                               memory=memory, 
                               tools=[search_tool, rag_tool], 
                               verbose=True) # verbose 수다스러운.. 출력을 더 많이 한다는 뜻!



In [55]:
agent_executor.invoke({'input': '방금 말한 내용이 2023 기준인데, 요즘엔 어떻게 됐대?'})



> Entering new AgentExecutor chain...

Invoking: `pdf_search` with `{'query': '삼성전자 최신 소식 2025'}`


저해상도 이미지의 고해상도 전환도 지원
n IT 전문지 테크리퍼블릭(TechRepublic)은 온디바이스 AI가 주요 기술 트렌드로 부상했다며, 
2024년부터 가우스를 탑재한 삼성 스마트폰이 메타의 라마(Llama)2를 탑재한 퀄컴 기기 및 구글 
어시스턴트를 적용한 구글 픽셀(Pixel)과 경쟁할 것으로 예상
☞ 출처 : 삼성전자, ‘삼성 AI 포럼’서 자체 개발 생성형 AI ‘삼성 가우스’ 공개, 2023.11.08.
삼성전자, ‘삼성 개발자 콘퍼런스 코리아 2023’ 개최, 2023.11.14.
TechRepublic, Samsung Gauss: Samsung Research Reveals Generative AI, 2023.11.08.

SPRi AI Brief |  
2023-12월호
10
삼성전자, 자체 개발 생성 AI ‘삼성 가우스’ 공개
n 삼성전자가 온디바이스에서 작동 가능하며 언어, 코드, 이미지의 3개 모델로 구성된 자체 개발 생성 
AI 모델 ‘삼성 가우스’를 공개
n 삼성전자는 삼성 가우스를 다양한 제품에 단계적으로 탑재할 계획으로, 온디바이스 작동이 가능한 
삼성 가우스는 외부로 사용자 정보가 유출될 위험이 없다는 장점을 보유
KEY Contents
£ 언어, 코드, 이미지의 3개 모델로 구성된 삼성 가우스, 온디바이스 작동 지원
n 삼성전자가 2023년 11월 8일 열린 ‘삼성 AI 포럼 2023’ 행사에서 자체 개발한 생성 AI 모델 
‘삼성 가우스’를 최초 공개
∙정규분포 이론을 정립한 천재 수학자 가우스(Gauss)의 이름을 본뜬 삼성 가우스는 다양한 상황에 
최적화된 크기의 모델 선택이 가능
∙삼성 가우스는 라이선스나 개인정보를 침해하지 않는 안전한 데이터를 통해 학습되었으며,

행사명
행사 주요 개요
CES 2024
- 미국 소비자기술 협회(C

{'input': '방금 말한 내용이 2023 기준인데, 요즘엔 어떻게 됐대?',
 'chat_history': [HumanMessage(content='오늘 기준으로 삼성전자 어떻게 바뀌었는지 찾아와', additional_kwargs={}, response_metadata={}),
  AIMessage(content="최근 삼성전자는 2025년 9월에 대한 다양한 소식을 발표하고 있어요. 주요 내용으로는 이번 달 4일에 열린 'IFA 2025' 행사에서 최신 갤럭시 S25 시리즈와 AI 탑재 프리미엄 태블릿을 공개했으며, IoT 보안 인증에서 최고 등급인 '스탠다드'를 획득한 2025년형 로봇청소기 출시 소식도 있습니다. 또한, 반도체 브랜드평판에서 1위를 기록하는 등 브랜드 평판 역시 지속적으로 강세를 보이고 있습니다. 이 외에도 삼성전자는 AI 및 디스플레이 기술 등 다양한 분야에서 활발한 활동을 이어가고 있습니다.", additional_kwargs={}, response_metadata={}),
  HumanMessage(content='pdf 기준으로 삼성전자 소식 알려줘', additional_kwargs={}, response_metadata={}),
  AIMessage(content='최근 삼성전자는 자체 개발 생성형 AI인 ‘삼성 가우스’를 공개했으며, 다양한 제품에 단계적으로 탑재할 계획입니다. ‘삼성 가우스’는 언어, 코드, 이미지 3가지 모델로 구성되어 있으며, 온디바이스에서 작동하여 외부에 사용자 정보가 유출되지 않는 안전성을 갖추고 있습니다. 이 AI는 텍스트 생성, AI 코딩 어시스턴트, 이미지 생성 및 수정, 고해상도 전환 등을 지원하며, 다양한 삼성 제품에 적용될 예정입니다. 또한, 삼성전자는 AI 관련 글로벌 행사인 CES 2024와 AIMLA 2024에 참가할 계획도 발표했습니다.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='방금 말한 내용